In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_digits

In [2]:
mnist = load_digits()

In [3]:
(mnist.data).shape, (mnist.target).shape

((1797, 64), (1797,))

In [4]:
len(mnist.data)

1797

In [5]:
len(mnist.target)

1797

In [6]:
def data_splitter(dataX, dataY):
  shuffled_indices = np.random.permutation(len(dataX))
  test_set_size = int(len(dataX) * 0.2)
  test_indices = shuffled_indices[:test_set_size]
  train_indices = shuffled_indices[test_set_size:]
  data_trainX, data_trainY = dataX[train_indices][:], dataY[train_indices]
  data_testX, data_testY =  dataX[test_indices][:], dataY[test_indices]
  return data_trainX, data_trainY, data_testX, data_testY

In [7]:
trnX, trnY, tstX, tstY = data_splitter(mnist.data, mnist.target)

In [8]:
len(trnX), len(trnY), len(tstX), len(tstY)

(1438, 1438, 359, 359)

In [27]:
def weight_initializer(trainX, trainY):
  weights = []
  indecies = []
  for l in range(10):
    indecies.append(int(np.argwhere(trainY == l)[0]))
    weights.append(list(trnX[int(np.argwhere(trainY == l)[0])][:]))
  trainX, trainY = np.delete(trainX, indecies, 0), np.delete(trainY, indecies)
  return np.array(weights), trainX, trainY  


def winner_distance(smpl, wght):
  comp = []
  for i in range(len(wght)):
    distance = np.sqrt(sum(np.power(smpl - wght[i], 2)))
    comp.append(distance)
  comp = np.array(comp)
  winner_index = int(np.argwhere(comp == np.min(comp))) 
  return winner_index


def update_weight(winner_label,actual_label,sample,weight,learning_rate):
  if winner_label == actual_label:
    weight[winner_label] = weight[winner_label] + learning_rate * (sample - weight[winner_label])
  else:
    weight[winner_label] = weight[winner_label] - learning_rate * (sample - weight[winner_label])
  return weight 

In [28]:
def lvq_trainer(trainX, trainY, weights, learning_rate):
  # Train LVQ
  for i in range(len(trainX)):
    ## def distance - winner
    winner = winner_distance(trainX[i], weights)
    ## def update
    weights = update_weight(winner, trainY[i], trainX[i], weights, learning_rate)
  return weights

In [29]:
def fit_lvq(trainX, trainY, learning_rate, epoche):
  # learning rate validation
  if (learning_rate >= 1) & (learning_rate < 0):
    print('Invalid learning rate')
    return None
  else:
    print('Learning rete is checked')  
  # initialize weights
  weights, trainX, trainY = weight_initializer(trainX, trainY)
  print('Weights are initialized')
  for i in range(epoche):
    weights = lvq_trainer(trainX, trainY, weights, learning_rate)
    print(f'epoche {i+1} ===> accuracy : {predictor(trainX, trainY, weights)}%')
  return weights

In [30]:
def predictor(testX, testY, wght):
  predicted = []
  for j in range(len(testX)):
    comp = []
    for i in range(len(wght)):
      distance = np.sqrt(sum(np.power(testX[j] - wght[i], 2)))
      comp.append(distance)
    
    comp = np.array(comp)
    winner_index = int(np.argwhere(comp == np.min(comp)))
    predicted.append(winner_index)
  accuracy = 100 * np.sum(np.array(predicted) == testY) / len(np.array(predicted) == testY)  
  
  return accuracy 

In [31]:
weights = fit_lvq(trnX, trnY, 0.007, 10)
predictor(tstX, tstY, weights)

Learning rete is checked
Weights are initialized
epoche 1 ===> accuracy : 77.45098039215686%
epoche 2 ===> accuracy : 80.812324929972%
epoche 3 ===> accuracy : 82.70308123249299%
epoche 4 ===> accuracy : 83.89355742296918%
epoche 5 ===> accuracy : 83.82352941176471%
epoche 6 ===> accuracy : 83.75350140056022%
epoche 7 ===> accuracy : 83.89355742296918%
epoche 8 ===> accuracy : 83.82352941176471%
epoche 9 ===> accuracy : 83.89355742296918%
epoche 10 ===> accuracy : 83.82352941176471%


83.008356545961

In [ ]:
# w_array = np.array([[ 0.,  0.,  3., 13.,  9.,  1.,  0.,  0.,  0.,  0., 13., 14., 15.,
#                       13.,  0.,  0.,  0.,  2., 14.,  1.,  2., 13.,  4.,  0.,  0.,  4.,
#                       8.,  0.,  0.,  5.,  8.,  0.,  0.,  4.,  8.,  0.,  0.,  4.,  8.,
#                       0.,  0.,  4., 10.,  0.,  0.,  5.,  8.,  0.,  0.,  0., 14., 11.,
#                       10., 14.,  5.,  0.,  0.,  0.,  4., 12., 13.,  9.,  0.,  0.],
#                     [ 0.,  0.,  0., 12., 10.,  0.,  0.,  0.,  0.,  0.,  0., 14., 16.,
#                       2.,  0.,  0.,  0.,  0.,  0., 13., 16.,  0.,  0.,  0.,  0.,  0.,
#                       0., 11., 16.,  3.,  0.,  0.,  0.,  0.,  0., 10., 16.,  3.,  0.,
#                       0.,  0.,  0.,  0., 11., 16.,  2.,  0.,  0.,  0.,  0.,  0., 14.,
#                       16.,  2.,  0.,  0.,  0.,  0.,  0., 11., 14.,  0.,  0.,  0.],
#                     [ 0.,  1., 15., 16., 10.,  0.,  0.,  0.,  0.,  7., 15., 10., 16.,
#                       0.,  0.,  0.,  0.,  4., 12.,  1., 16.,  4.,  0.,  0.,  0.,  0.,
#                       2.,  3., 16.,  1.,  0.,  0.,  0.,  0.,  0.,  4., 15.,  0.,  0.,
#                       0.,  0.,  0.,  0., 11., 12.,  0.,  0.,  0.,  0.,  0., 11., 16.,
#                       14., 14., 15.,  3.,  0.,  1., 15., 16., 16., 16., 16.,  5.],
#                     [ 0.,  1.,  9., 15., 16.,  6.,  0.,  0.,  0., 13., 15., 10., 16.,
#                       11.,  0.,  0.,  0.,  5.,  3.,  4., 16.,  7.,  0.,  0.,  0.,  0.,
#                       0.,  8., 16.,  7.,  0.,  0.,  0.,  0.,  0.,  1., 13., 15.,  5.,
#                       0.,  0.,  0.,  0.,  0.,  2., 13., 11.,  0.,  0.,  0., 12.,  5.,
#                       3., 13., 14.,  0.,  0.,  0., 10., 16., 16., 14.,  5.,  0.],
#                     [ 0.,  0.,  0., 12., 12.,  0.,  0.,  0.,  0.,  0.,  5., 16.,  4.,
#                       0.,  0.,  0.,  0.,  1., 14., 11.,  0.,  0.,  0.,  0.,  0.,  6.,
#                       16.,  3.,  2.,  0.,  0.,  0.,  0., 13., 12.,  8., 12.,  0.,  0.,
#                       0.,  0., 15., 16., 15., 16., 13.,  4.,  0.,  0.,  4.,  9., 14.,
#                       16.,  7.,  0.,  0.,  0.,  0.,  0., 11., 13.,  0.,  0.,  0.],
#                     [ 0.,  2., 12., 13., 16., 16.,  4.,  0.,  0., 11., 16., 13.,  7.,
#                       4.,  1.,  0.,  0., 13., 14.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
#                       15., 12.,  0.,  0.,  0.,  0.,  0.,  0.,  6., 16.,  3.,  0.,  0.,
#                       0.,  0.,  0.,  0., 13.,  7.,  0.,  0.,  0.,  0.,  3.,  5., 16.,
#                       7.,  0.,  0.,  0.,  0.,  3., 13., 15.,  0.,  0.,  0.,  0.],
#                     [ 0.,  0.,  0., 11., 13.,  5.,  0.,  0.,  0.,  0.,  3., 16., 13.,
#                       3.,  0.,  0.,  0.,  0., 10., 16.,  2.,  0.,  0.,  0.,  0.,  4.,
#                       16., 16., 13.,  7.,  0.,  0.,  0.,  4., 16., 11.,  8., 16.,  2.,
#                       0.,  0.,  0., 15.,  8.,  0., 15.,  6.,  0.,  0.,  0.,  9., 14.,
#                       4., 15.,  4.,  0.,  0.,  0.,  1., 10., 16., 11.,  1.,  0.],
#                     [ 0.,  0.,  9., 16., 16., 16.,  5.,  0.,  0.,  1., 14., 10.,  8.,
#                       16.,  8.,  0.,  0.,  0.,  0.,  0.,  7., 16.,  3.,  0.,  0.,  3.,
#                       8., 11., 15., 16., 11.,  0.,  0.,  8., 16., 16., 15., 11.,  3.,
#                       0.,  0.,  0.,  2., 16.,  7.,  0.,  0.,  0.,  0.,  0.,  8., 16.,
#                       1.,  0.,  0.,  0.,  0.,  0., 13., 10.,  0.,  0.,  0.,  0.],
#                     [ 0.,  0.,  6., 14., 16.,  6.,  0.,  0.,  0.,  6., 16., 16.,  8.,
#                       15.,  0.,  0.,  0.,  7., 14., 14., 12., 14.,  0.,  0.,  0.,  0.,
#                       13., 10., 16.,  6.,  0.,  0.,  0.,  0.,  4., 16., 10.,  0.,  0.,
#                       0.,  0.,  0., 11., 13., 16.,  2.,  0.,  0.,  0.,  0., 15.,  5.,
#                       15.,  4.,  0.,  0.,  0.,  0.,  8., 16., 15.,  1.,  0.,  0.],
#                     [ 0.,  0., 12., 16., 15.,  6.,  0.,  0.,  0.,  0., 15., 12.,  7.,
#                       15.,  1.,  0.,  0.,  1., 15., 15.,  7., 16.,  4.,  0.,  0.,  1.,
#                       12., 16., 16., 14.,  1.,  0.,  0.,  0.,  0.,  4., 10., 13.,  0.,
#                       0.,  0.,  0.,  0.,  0.,  1., 15.,  3.,  0.,  0.,  0.,  3.,  0.,
#                       2., 16.,  6.,  0.,  0.,  0., 13., 16., 16., 15.,  1.,  0.]])
# s_array = trnX[0]
# print(s_array)
# print(trnY[0])

array([[ 2,  6, 12, 30],
       [ 7,  3,  4,  9]])